# RISK AND UNCERTAINTY ANALYSIS IN RESERVOIR DEVELOPMENT

In our Oil and Gas Industry all over cost from starting of exploration to production stage very high compare to other industry. Also during the exploration stage, we don’t have more information regarding reservoir. Based on different property we found that this reservoir has oil and gas. So, next step is that we want to know how much barrel of oil is present in reservoir. For finding OOIP we use different property like Porosity, Saturation of water, Gross rock volume and Net pay thickness. But we are not directly got these properties. First we find the rock and fluid physical property then we create this to our interested property.

 To find this property we are using well logging, Seismic survey, well testing. We know that reservoir is fully uncertain and this all method is giving uncertain property. So, if we are use this property to find OOIP calculation then it will give Wrong OOIP value. Also, While Simulation model making, we are using this property. So, this model is also uncertain and is we are predicting something from this then it will give wrong result. So, there is huge risk from investor side. Because investor see only OOIP value according to probability of occurrence. 
 
From above data if we see then there is a huge risk in investor side. So, here we have use Monte Carlo simulation for the generating the expected OOIP value. Using Monte Carlo simulation, we are increasing a number of experiments to get an expected output. We have taken an input data and made a python code for depletion drive reservoir case and we generated reservoir performance data. And then we made a production profile case. And we made a worst-case scenario and best-case scenario. And then investor sees these scenarios and takes a decision that I have to invest money in this field or not.


# Python code for monte carlo simulation for IOIP

In [8]:
import math as m
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
"""
Here Data is imported from data.txt file and converted 
this data into array form
"""
Data = np.genfromtxt("Reserve_Data.txt",delimiter=',')
FVF = Data[0,0]

Porosity = []
Saturation = []
Height = []
Area = []
IOIP_L = []

def IOIP(A, h, So, Phi, FVF):
    """
    :param A: Area(m^2)
    :param h: Thickness (m)
    :param Sw: Saturation of oil
    :param Phi: Porosity
    :param FVF: Formation Volume factor
    :return: Initial Oil in place (m^3)
    """

    Oil_in_place = (A*h*So*Phi)/FVF
    return Oil_in_place

# Sampling the Data (Random Variable)
for i in range(100):
    rd = np.random.random()  # Random Number

    # Random Variable 1 : Porosity
    """
    To generate 100 Random Variable For Porosity
    Triangle Distribution
    """
    if rd <= ((Data[3,2] -Data[3,1])/(Data[3,0]-Data[3,1])):
        X = Data[3,1] + m.sqrt(rd*(Data[3,2] -Data[3,1])*(Data[3,0]-Data[3,1]))
    elif rd >= ((Data[3,2] -Data[3,1])/Data[3,0]-Data[3,1]):
        X = Data[3,0] - m.sqrt((1-rd)*(Data[3,0]-Data[3,2] )*(Data[3,0]-Data[3,1]))
    Porosity.append(X)

    # Random Variable 2 : Saturation
    """
    To generate 100 Random Variable for the Saturation
    Triangle Distribution
    """
    if rd <= ((Data[4,2] -Data[4,1])/(Data[4,0]-Data[4,1])):
        X1 = Data[4,1] + m.sqrt(rd*(Data[4,2] -Data[4,1])*(Data[4,0]-Data[4,1]))
    elif rd >= ((Data[4,2] -Data[4,1])/Data[4,0]-Data[4,1]):
        X1 = Data[4,0] - m.sqrt((1-rd)*(Data[4,0]-Data[4,2] )*(Data[4,0]-Data[4,1]))
    Saturation.append(X1)

    # Random Variable 3: Height (meter)
    """
    To generate 100 Random Variable for the Height
    Triangle Distribution
    """

    if rd <= ((Data[5,2] -Data[5,1])/(Data[5,0]-Data[5,1])):
        X2 = Data[5,1] + m.sqrt(rd*(Data[5,2] -Data[5,1])*(Data[5,0]-Data[5,1]))
    elif rd >= ((Data[5,2] -Data[5,1])/Data[5,0]-Data[5,1]):
        X2 = Data[5,0] - m.sqrt((1-rd)*(Data[5,0]-Data[5,2] )*(Data[5,0]-Data[5,1]))
    Height.append(X2)

    # Random Variable 4 : Area (m^2)
    """
    To generate 100 Random Variable for the area
    Triangle Distribution
    """

    if rd <= ((Data[2,2] -Data[2,1])/(Data[2,0]-Data[2,1])):
        X3 = Data[2,1] + m.sqrt(rd*(Data[2,2] -Data[2,1])*(Data[2,0]-Data[2,1]))
    elif rd >= ((Data[2,2] -Data[2,1])/Data[2,0]-Data[2,1]):
        X3 = Data[2,0] - m.sqrt((1-rd)*(Data[2,0]-Data[2,2] )*(Data[2,0]-Data[2,1]))
    Area.append(X3)

    """
    Find oil initial in place
    """
    Oil_in_place = IOIP(X3,X2,X1,X,FVF)
    IOIP_L.append(Oil_in_place)

dict1 = {"porosity":Porosity,
         "Saturation":Saturation,
         "Height(m)":Height,
         "Area(m^2)":Area,
         "Oil in Place(m^3)":IOIP_L}
print("Dict1",dict1)
df = pd.DataFrame(dict1)
print(df)
#df.to_excel("Reserve_Calculation.xlsx")

x = IOIP_L
plt.hist(x,10,ec="red")
plt.xlabel("OIIP (m^3)")
plt.ylabel("count")
plt.show()


# Python code Application of monte carlo in depletion drive reservoir

In [ ]:
import math as m
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

"""
Here Data is imported from data.txt file and converted 
this data into array form
"""
Data1 = np.genfromtxt("Depletion_Data.txt",delimiter=',')

"""
Here Data is imported from data.txt file and converted 
this data into array form
"""
Data2 = np.genfromtxt("Application_Data.txt",delimiter=',')


"""
Using the Corelation to find the PVT properties
Making all the Co-relation as the Function of the Pressure
"""


def Rs_GOR(P):
    """
    This function gives value of Solution GOR
    :param P: Pressure (Psi)
    :return: solution GOR (SCF/STB)
    """
    x = (7.916*(0.0001)*Data1[0][0]**1.541) - 4.561*(0.00001)*(Data1[0][1]-460)**1.3911

    Rs = (((P/112.727)+ 12.340)*((Data1[0][2])**0.8439)*(10**x))**1.73184

    return Rs



def OIL_FVF(P):
    """
    Standing Corelation
    This function gives value of oil formation volume factor
    :param Rs: Solution GOR (Scf/STB)
    :return: Oil formation volume factor (bbl/STB)
    """
    Rs = Rs_GOR(P)
    Bo = 0.9759 + 0.000120*(Rs*((Data1[0][2]/Data1[0][3])**0.5)+1.25*(Data1[0][1]-460))**1.2
    return Bo

def Muo_Oil(P):
    """
    This function gives value of the oil at  pressure
    :param Rs: Gas solubiolity (SCF/STB)
    :return: Saturated  oil viscosity (cp)
    """
    Rs = Rs_GOR(P)
    Z = 3.0324 - 0.020238*Data1[0][0]
    Y = 10**Z
    X = Y*(Data1[0][1]-460)**(-1.163)
    Muo_d = 10**X - 1    #Dead oil viscosity (P = 14.7 psi)

    a = 10.715 * (Rs + 100) ** (-0.515)
    b = 5.44 * (Rs + 150) ** (-0.338)
    Mu_ob = a * ((Muo_d) ** b)   #Viscosity at Bubble point Pressure
    return Mu_ob




def viscosity_gas(P):
    """
    This Function gives the value of the Viscosity of gas
    :param P: Reservoir Pressure (Psi)
    :return: Viscosity (cP)
    """
    Mw = Data1[0][2]*28.96                           # Molecular of weight of gas
    Z = Z_factor(P)
    Rho_G = (1.4935*1e-3*P*Mw)/(Z*Data1[0][1])        # Density of gas(in gm/cc)
    K = ((9.4+0.02*Mw)*(Data1[0][1]**1.5))/(209+19*Mw+Data1[0][1])
    X = 3.5+(986/Data1[0][1])+0.01*Mw
    Y = 2.4-0.2*X
    Mug = 1e-4*K*m.exp(X*(Rho_G)**Y)        # Visocity (In cP)
    return Mug

def Z_factor(P):
    """
    This function Gas compresibility factor
    :param P: Pressure (Psi)
    :return:  z (Compresiblity Factor)
    """
    if Data1[0][2] < 0.75:
        # Here gas consider as dry gas
        Tc = 168 + 325 * Data1[0][2] - 12.5 * (Data1[0][2]) ** 2  # In degree Renkine unit
        Pc = 677 + 15 * Data1[0][2] - 37.5 * (Data1[0][2]) ** 2  # In psia unit
    else:
        # Here gas consider as wet gas
        # Here specific gravity is more than the 0.75
        Tc = 187 + 330 * Data1[0][2] - 71.5 * (Data1[0][2]) ** 2  # In degree Renkine unit
        Pc = 706 + 51.7 * Data1[0][2] - 11.1 * (Data1[0][2]) ** 2  # In psia unit


    Tr = Data1[0][1] / Tc            # Reduce Temperature
    Pr = P / Pc           # Reduce Pressure

    """
    The Hall-Yarboprough Metho to find the Z-Factor
    """


    tr = Tr
    pr = Pr
    t = 1 / tr
    X1 = -0.06125 * t * pr * (m.exp((-1.2) * (1 - t) ** 2))
    X2 = 14.76 * t - 9.76 * (t ** 2) + 4.58 * (t ** 3)
    X3 = 90.7 * t - 242.2 * (t ** 2) + 42.4 * (t ** 3)
    X4 = 2.18 + 2.82 * t

    y = 0.0125 * pr * t * (m.exp(-1.2 * (1 - t) ** 2))

    """
    Newton Rapson Method Use to solve the Non-linear
    equation 
    """

    for i in range(10000):
        fny = X1 + (y + y ** 2 + y ** 3 + y ** 4) / ((1 - y) ** 3) - X2 * (y ** 2) + X3 * (y ** X4)
        dfny = ((1+4*y+4*(y**2)-4*(y**3)+y**4)/((1-y)**4)) - (2*X2*y) + X3*X4*(y**(X4-1))
        y_new = y - (fny / dfny)
        if abs(y_new - y) < 1e-12:
            break
        y = y_new

    y1 = y_new

    z = (0.06125 * pr * t * m.exp(-1.2 * (1 - t) ** 2)) / y1

    return z

def Inverse_FVF_gas(P):
    """
    This function guives value of Gas formation volume factor
    :param P: Pressure (Psi)
    :return: Gas Formation volume factor(bbl/scf)
    """

    Z_fac = Z_factor(P)
    Bg = (0.005035*Z_fac*Data1[0][1])/P
    return 1/Bg

def FVF_gas(P):
    """
    This function guives value of Gas formation volume factor
    :param P: Pressure (Psi)
    :return: Gas Formation volume factore (cf/scf)
    """

    Z_fac = Z_factor(P)
    Bg = (0.005035*Z_fac*Data1[0][1])/P
    return Bg

def Rel_K(Sg):
    """
    Using Corile's Corelation to Get Relative Permeability
    :param Sg: Saturation of Gas
    :return: Kro
    """
    Sg_star = Sg / (1 - Data1[1][0])
    Kro = (1 - Sg_star) ** 4
    Krg = (Sg_star ** 3) * (2 - Sg_star)
    return Kro


def Rel_Kg(Sg):
    Sg_star = Sg / (1 - Data1[1][0])
    Kro = (1-Sg_star)**4
    Krg = (Sg_star**3)*(2-Sg_star)
    return Krg


def Ratio_Rel_K(Sg):
    Sg_star = Sg/(1-Data1[1][0])
    Kro = (1-Sg_star)**4
    Krg = (Sg_star**3)*(2-Sg_star)
    return Krg/Kro


def dRs_by_dP(P):
    """
    This Function gives derivative of the Gas solubility
    :param P: Pressuer(Psi)
    :return: derivative
    """
    h = 0.00000001
    return (Rs_GOR(P+h) - Rs_GOR(P))/h

def dBo_by_dP(P):
    """
    This function gives the derivative of the oil formation volume factor
    :param P: Pressure(Psi)
    :return: Derivative of the Bo
    """
    h = 0.00000001
    Rs_1 = Rs_GOR(P)
    Rs_2 = Rs_GOR(P+h)
    return (OIL_FVF(Rs_2) - OIL_FVF(Rs_1))/h

def dP_by_dBg(P):
    """
    This Function give the Derivative of the Inverse the Bg
    :param P: Pressure(Psi)
    :return: Gives d(1/Bg)/dP
    """
    h = 0.00000001
    return (Inverse_FVF_gas(P+h)-Inverse_FVF_gas(P))/h

def X_of_P(P):
    return (FVF_gas(P) / OIL_FVF(P)) * dRs_by_dP(P)

def Y_of_P(P):
    return (1/OIL_FVF(P))*(Muo_Oil(P)/viscosity_gas(P))*dBo_by_dP(P)*10

def Z_of_P(P):
    return FVF_gas(P) * dP_by_dBg(P)


"""
Following the Stepes which are written in the Tarik-Ahemad
Book 
"""

# Step: 3 Prepare the Graph between the X_of_P,Y_of_P and Z_of_P and P

X = []
Pr = []
Y = []
Z = []
for i in range(2500,500,-100):
    P = i
    X_of_P1 = X_of_P(P)
    Y_of_P1 = Y_of_P(P)
    Z_of_P1 = Z_of_P(P)
    Pr.append(i)
    X.append(X_of_P1)
    Y.append(Y_of_P1)
    Z.append(Z_of_P1)


dict1 = {"Pressure":Pr,
         "X_of_P":X,
         "Y_of_P":Y,
         "Z_of_P":Z}

df = pd.DataFrame(dict1)
print(df)

plt.plot(df["Pressure"],df["X_of_P"],label = 'X(P)')

plt.plot(df["Pressure"],df["Y_of_P"],label="Y(P)")
plt.plot(df["Pressure"],df["Z_of_P"],label = "Z(P)")
plt.legend()
plt.xlabel("Pressure(Psi)")
plt.ylabel("X(P),Y(P)& Z(P)")
plt.grid()
plt.show()

#Step_6: solve the Mustak Equation
P_i = Data1[1][3]
Sgi = Data1[1][2]
Sg = Sgi
Soi = Data1[1][1]
So = Soi

def del_So_by_del_P(P,So,Sg):
    return ((So*X_of_P(P)) + (So*Ratio_Rel_K(Sg)*Y_of_P(P)) + (1-So-Data1[1][0])*Z_of_P(P))/(1+(Muo_Oil(P)/viscosity_gas(P))*Ratio_Rel_K(Sg))

Pre_Psi = []
Np_STB = []
Gp_SCF = []
Saturation_Oil = []
Saturation_Gas = []
Bo = []
Relative_K = []
I_GOR = []
Krg = []
So_by_Bo = []

"""
    Random Variable 2 : IOIP(STB)
    Generating 100 Variable of the IOIP
    """
rd = np.random.random()

if rd<=((Data2[1][2]-Data2[1][1])/(Data2[1][0]-Data2[1][1])):
    IOIP1 = Data2[1][1] + m.sqrt(rd*(Data2[1][2]-Data2[1][1])*(Data2[1][0]-Data2[1][1]))
elif rd>= ((Data2[1][2]-Data2[1][1])/(Data2[1][0]-Data2[1][1])):
    IOIP1 = Data2[1][0]- m.sqrt((1-rd)*(Data2[1][0]-Data2[1][2])*(Data2[1][0]-Data2[1][1]))

# Cumulative Oil Production
P = 2500
for i in range(2500,700,-1):

    P_new = i-1
    dSo_by_dP_1 = del_So_by_del_P(P, So, Sg)  #This term Constant For Every Iteration
    So_new = Soi - (P - P_new) * del_So_by_del_P(P, So, Sg)
    Sg_1 = 1 - Data1[1][0] - So_new
    dSo_by_dP_2 = del_So_by_del_P(P_new, So_new, Sg_1)

    dSo_by_dP_avg = (dSo_by_dP_1 + dSo_by_dP_2) / 2

    So_2 = Soi - (P - P_new) * dSo_by_dP_avg

    Sg_f = 1 - Data1[1][0] - So_2
    K = Rel_K(Sg_f)
    Rel_Krg = Rel_Kg(Sg_f)
    Np = IOIP1 * (1 - ((OIL_FVF(P) / OIL_FVF(P_new)) * (So_2 / (1 - Data1[1][0]))))

    #GOR_1 = Rs_GOR(P) + Ratio_Rel_K(Sg) * (Muo_Oil(P) / viscosity_gas(P))
    GOR_2 = Rs_GOR(P_new) + (Ratio_Rel_K(Sg_f) * (Muo_Oil(P_new) / viscosity_gas(P_new)))

    #Gp = ((GOR_1 + GOR_2) / 2) * Np
    Pre_Psi.append(P_new)
    Np_STB.append(Np)
    #Gp_SCF.append(Gp)
    Saturation_Oil.append(So_2)
    Saturation_Gas.append(Sg_f)
    Bo.append(OIL_FVF(P_new))
    So_by_Bo.append(So_2/OIL_FVF(P_new))
    Relative_K.append(K)
    Krg.append(Rel_Krg)
    I_GOR.append(GOR_2)

    #So = So_2
    #Sg = Sg_f



dict2 = {"Pressure(Psi)":Pre_Psi,
         "Np(STB)":Np_STB,
         "So":Saturation_Oil,
         "Sg":Saturation_Gas,
         "Bo":Bo,
         "Relative":Relative_K,
         "Krg":Krg,
         "So/Bo":So_by_Bo,
         "I_GOR1(SCF/STB)":I_GOR}

df2 = pd.DataFrame(dict2)
#df2.to_excel("Np_&_data_r1100.xlsx")
print(df2)

"""
plt.plot(Saturation_Gas,Krg)
plt.title("Krg Vs Sg")
plt.xlabel("Sg")
plt.ylabel("Krg")
plt.grid()
plt.show()

plt.plot(Pre_Psi,Krg)
plt.title("Krg Vs P")
plt.xlabel("Pressure(Psi)")
plt.ylabel("Krg")
plt.grid()
plt.show()

plt.plot(Pre_Psi,Np_STB)
plt.title("Cumulative Oil Production")
plt.xlabel("Pressure(Psi)")
plt.ylabel("Np(STB)")
plt.grid()
plt.show()

plt.plot(Pre_Psi,Saturation_Oil)
plt.title("Saturation")
plt.xlabel("Pressure(Psi)")
plt.ylabel("So")
plt.grid()
plt.show()

plt.plot(Pre_Psi,I_GOR)
plt.title("I_GOR")
plt.xlabel("Pressure(Psi)")
plt.ylabel("I_GOR(SCF/STB)")
plt.grid()
plt.show()

"""

"""
Calculation of the Reservoir Pressure
"""

def R_H_S(Np):
    return ((1-Data1[1][0])/OIL_FVF(Data1[1][3]))*(1-(Np/Data1[0][0]))

def Reservoir_P(Np):

    RHS = Np
    i = 0
    while True:
        LHS = df2["Np(STB)"][i]
        if abs(LHS-RHS)<6000:
            #print(LHS-RHS)
            P = df2["Pressure(Psi)"][i]
            #print("P Np",P,Np)
            break
        i = i+1
    return P


# Combination of Darcy and MBE

def Re(A):
    """
    This function gives the value of Draniage radius
    :param A: Area
    :return: Radius (in ft)
    """
    return m.sqrt(A/m.pi)

def Qmax():
    """
    This Function Gives Maximum Flow Rate
    :return: Qmax (STB/day)
    """
    return (0.00708*Data1[2][2]*Rel_K(Data1[1][2])*Data1[1][3])/(Muo_Oil(Data1[1][3])*OIL_FVF(Data1[1][3])*(deno))


def FBHP(Qo,P,A,KH):
    """
    Gives the Flowing bottom hole Pressure
    :param Qo: Oil flow Rate (STB/day)
    :param P: Pressure(Psi)
    :return: Flowing Bottom Hole Pressure(psi)
    """
    deno = m.log(Re(A) / Data1[2][3], m.e) - 0.75 + Data1[3][0]
    Pwf = P - ((141.2*Qo*Muo_Oil(P)*OIL_FVF(P)*deno)/KH)
    return Pwf

FBHP_L = []
Pre_Reservoir_L = []
Q_L = []
Np_L = []
Days_L = []
P = 2500


for i in range(1):

    """
       Random VAriable 1: Qo (STB/day)
       Generating 100 VAriable of the Qo
       """

    if rd <= ((Data2[3][2] - Data2[3][1]) / (Data2[3][0] - Data2[3][1])):
        Qo1 = Data2[3][1] + m.sqrt(rd * (Data2[3][2] - Data2[3][1]) * (Data2[3][0] - Data2[3][1]))
    elif rd >= ((Data2[3][2] - Data2[3][1]) / (Data2[3][0] - Data2[3][1])):
        Qo1 = Data2[3][0] - m.sqrt((1 - rd) * (Data2[3][0] - Data2[3][2]) * (Data2[3][0] - Data2[3][1]))


    """
    Random Variable 2: Area(ft^2)
    Generating 100 rnadom variable of the Area
    """

    if rd<=((Data2[0][2]-Data2[0][1])/(Data2[0][0]-Data2[0][1])):
        A1 = Data2[0][1] + m.sqrt(rd*(Data2[0][2]-Data2[0][1])*(Data2[0][0]-Data2[0][1]))
    elif rd>= ((Data2[0][2]-Data2[0][1])/(Data2[0][0]-Data2[0][1])):
        A1 = Data2[0][0]- m.sqrt((1-rd)*(Data2[0][0]-Data2[0][2])*(Data2[0][0]-Data2[0][1]))

    """
    Random Variable 3: KH(mD-ft)
    Generating 100 Variable of the KH
    """
    if rd <= ((Data2[2][2] - Data2[2][1]) / (Data2[2][0] - Data2[2][1])):
        KH1 = Data2[2][1] + m.sqrt(rd * (Data2[2][2] - Data2[2][1]) * (Data2[2][0] - Data2[2][1]))
    elif rd >= ((Data2[2][2] - Data2[2][1]) / (Data2[2][0] - Data2[2][1])):
        KH1 = Data2[2][0] - m.sqrt((1 - rd) * (Data2[2][0] - Data2[2][2]) * (Data2[2][0] - Data2[2][1]))

    Q = Qo1
    i = 1

    while True:
        FBHP_Pwf1 = FBHP(Q,P,A1,KH1)
        if FBHP_Pwf1<711:

            while True:
                Q = Q*0.99
                FBHP_Pwf1 = FBHP(Q, P,A1,KH1)
                if FBHP_Pwf1>711:
                    break
        FBHP_L.append(FBHP_Pwf1)
        #Q_L.append(Q)
        if i == 1:
            Np = 0
        Np = Np + Q*365
        #print("Np",Np)
        #print("Q", Q)


        if Np>IOIP1*0.15:
            break

        Np_L.append(Np)
        Pr = Reservoir_P(Np)
        Pre_Reservoir_L.append(Pr)
        P = Pr
        Q_L.append(Q)
        Days_L.append(i*365)
        i = i+1
        if P<800:
            break



    dict_d = {"Days":Days_L,
              "Q(STB/days)":Q_L,
              "Np":Np_L,
              "Pr(Psi)":Pre_Reservoir_L,
              "FBHP":FBHP_L
              }
    df_frame = pd.DataFrame(dict_d)
    df_frame.to_excel("Data_PP.xlsx")
    print("IOIP", IOIP1)
    print("Q", "A", "KH", Qo1, A1, KH1)

    plt.plot(Days_L,Q_L)
    plt.title("Flow rate Vs Days")
    plt.xlabel("Days")
    plt.ylabel("Q(STB/day)")
    plt.grid()
    plt.show()

    plt.plot(Days_L,Pre_Reservoir_L,label="Pre")
    plt.plot(Days_L,FBHP_L,label="FBHP")
    plt.title("Pressure Vs Time")
    plt.xlabel("Days")
    plt.ylabel("Pressure(Psi)")
    plt.grid()
    plt.legend()
    plt.show()


## Code For generating simulations 

i have recorded each scenario and then made a xlsx file of that scenario

In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(16, 10), dpi=80)

plt.plot( h1['Q1'], color='red', label='Q1',marker='o', linewidth=2)
plt.plot( h1['Q2'], color='green', label='Q2',marker='o', linewidth=2)
plt.plot( h1['Q3'], color='blue', label='Q3',marker='o', linewidth=2)
plt.plot( h1['Q4'], color='black', label='Q4',marker='o', linewidth=2)
plt.plot( h1['Q5'], color='purple', label='Q5',marker='o', linewidth=2)
plt.plot( h1['Q6'], color='pink', label='Q6',marker='o', linewidth=2)
plt.plot( h1['Q7'], color='orange', label='Q7',marker='o', linewidth=2)
plt.plot( h1['Q8'], color='m', label='Q8',marker='o', linewidth=2)
plt.plot( h1['Q9'], color='magenta', label='Q9',marker='o', linewidth=2)
plt.plot( h1['Q10'], color='brown', label='Q10',marker='o', linewidth=2)
plt.plot( h1['Q11'], color='cyan', label='Q11',marker='o', linewidth=2)
plt.plot( h1['Q12'], color='darkred', label='Q12',marker='o', linewidth=2)
plt.plot( h1['Q13'], color='darkblue', label='Q13',marker='o', linewidth=2)
plt.plot( h1['Q14'], color='steelblue', label='Q14',marker='o', linewidth=2)
plt.plot( h1['Q15'], color='darkgreen', label='Q15',marker='o', linewidth=2)
plt.plot( h1['Q16'], color='orange', label='Q16',marker='o', linewidth=2)
plt.plot( h1['Q17'], color='black', label='Q17',marker='o', linewidth=2)
plt.plot( h1['Q18'], color='purple', label='Q18',marker='o', linewidth=2)
plt.plot( h1['Q19'], color='teal', label='Q19',marker='o', linewidth=2)
plt.plot( h1['Q20'], color='cyan', label='Q20',marker='o', linewidth=2)
plt.plot( h1['Q21'], color='orchid', label='Q21',marker='o', linewidth=2)
plt.plot( h1['Q22'], color='pink', label='Q22',marker='o', linewidth=2)
plt.plot( h1['Q23'], color='lightgreen', label='Q23',marker='o', linewidth=2)
plt.plot( h1['Q24'], color='maroon', label='Q24',marker='o', linewidth=2)
plt.plot( h1['Q25'], color='coral', label='Q25',marker='o', linewidth=2)

plt.xlabel('Time(Days)',fontsize=20)
plt.ylabel('Flowrate(BOPD)',fontsize=20)
plt.title('Various simulation for Flowrate vs time')
plt.grid(True)
plt.legend(prop={'size': 9})
plt.show()